In [2]:
#
# 檔名: generate_captions_v3.py (版本 3.0 - 增加即時進度顯示與速度優化)
#
import os
import pandas as pd
import torch
from PIL import Image
from transformers import AutoProcessor, LlavaForConditionalGeneration, BitsAndBytesConfig
from tqdm import tqdm
import glob
import time

# --- 1. 組態設定 ---
BASE_DATA_DIR = "./" 
ROAD_TRAIN_CSV = os.path.join(BASE_DATA_DIR, 'road_train.csv')
FREEWAY_TRAIN_CSV = os.path.join(BASE_DATA_DIR, 'freeway_train.csv')
OUTPUT_ROAD_TRAIN_CSV = os.path.join(BASE_DATA_DIR, 'road_train_with_captions.csv')
OUTPUT_FREEWAY_TRAIN_CSV = os.path.join(BASE_DATA_DIR, 'freeway_train_with_captions.csv')
MODEL_ID = "llava-hf/llava-1.5-7b-hf"
PROMPT = (
    "You are an expert in traffic safety analysis. "
    "Describe this dashcam image in one sentence focusing on potential hazards, "
    "such as nearby vehicles' actions, pedestrians, road conditions, and the ego car's movement. "
    "If no immediate hazard is visible, describe the general traffic scene."
)
# --- 優化設定 ---
# 將圖片寬度縮小到這個尺寸，可以大幅加速。設為 None 則使用原始尺寸。
RESIZE_WIDTH = 800

# --- 2. 模型載入 ---
def load_model_and_processor():
    print("正在載入模型，請稍候...")
    if not torch.cuda.is_available():
        raise ConnectionError("沒有偵測到 CUDA GPU。此模型需要在 GPU 上運行。")
    print(f"偵測到 CUDA 設備: {torch.cuda.get_device_name(0)}")
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = LlavaForConditionalGeneration.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        quantization_config=quantization_config,
    )
    processor = AutoProcessor.from_pretrained(MODEL_ID)
    print("模型載入完成！")
    return model, processor

# --- 3. 圖片描述生成 (加入圖片縮小優化) ---
def generate_caption_for_image(model, processor, image_path, prompt):
    try:
        raw_image = Image.open(image_path).convert('RGB')

        # ====> 優化點：如果設定了 RESIZE_WIDTH，就縮小圖片 <====
        if RESIZE_WIDTH:
            width, height = raw_image.size
            if width > RESIZE_WIDTH:
                new_width = RESIZE_WIDTH
                new_height = int(new_width * height / width)
                raw_image = raw_image.resize((new_width, new_height))
        
        final_prompt = f"USER: <image>\n{prompt}\nASSISTANT:"
        inputs = processor(text=final_prompt, images=raw_image, return_tensors="pt").to("cuda", torch.float16)
        output = model.generate(**inputs, max_new_tokens=100, do_sample=False)
        decoded_text = processor.decode(output[0], skip_special_tokens=True)
        caption = decoded_text.split("ASSISTANT:")[-1].strip()
        return caption
    except Exception as e:
        print(f"處理圖片 {image_path} 時發生錯誤: {e}")
        return "Error generating caption."

# --- 4. 資料集處理主函式 (加入內部進度打印) ---
def process_dataset(model, processor, csv_path, data_type, output_csv_path, is_test_set=False):
    if not os.path.exists(csv_path):
        print(f"警告：找不到 CSV 檔案 {csv_path}。跳過此資料集處理。")
        return

    print(f"\n--- 開始處理 {data_type} ({'測試集' if is_test_set else '訓練集'}) ---")
    df = pd.read_csv(csv_path)
    
    # 如果輸出檔案已存在，可以選擇從上次中斷的地方繼續
    if os.path.exists(output_csv_path):
        print(f"偵測到已存在的輸出檔 {output_csv_path}。")
        output_df = pd.read_csv(output_csv_path)
        # 確保欄位存在
        if 'captions' not in output_df.columns:
            output_df['captions'] = ""
        if 'image_count' not in output_df.columns:
            output_df['image_count'] = 0
    else:
        output_df = df.copy()
        output_df['image_count'] = 0
        output_df['captions'] = ""

    data_folder = 'test' if is_test_set else 'train'
    
    # 使用 tqdm 顯示外層迴圈進度
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc=f"處理 {data_type} 進度"):
        # 檢查是否已經處理過
        # pd.isna checks for NaN, None, etc.
        if 'captions' in output_df.columns and index < len(output_df) and pd.notna(output_df.at[index, 'captions']) and output_df.at[index, 'captions'] != "":
            continue

        file_name = row['file_name']
        image_dir = os.path.join(BASE_DATA_DIR, data_type, data_folder, file_name)

        if not os.path.isdir(image_dir):
            print(f"找不到資料夾: {image_dir}，跳過。")
            output_df.at[index, 'captions'] = "Directory not found."
            continue
        
        image_files = sorted(glob.glob(os.path.join(image_dir, '*.[jJ][pP][gG]')))

        output_df.at[index, 'image_count'] = len(image_files)

        if not image_files:
            output_df.at[index, 'captions'] = "No images found."
            continue

        all_captions = []
        start_time = time.time() # 計時開始
        
        # ====> 內層迴圈：處理單一資料夾的所有圖片 <====
        for img_idx, image_path in enumerate(image_files):
            # ====> 這就是新的進度顯示！ <====
            print(f"  -> 正在處理 {file_name} 的第 {img_idx + 1}/{len(image_files)} 張圖片: {os.path.basename(image_path)}")
            
            caption = generate_caption_for_image(model, processor, image_path, PROMPT)
            all_captions.append(caption)

        end_time = time.time() # 計時結束
        elapsed_time = end_time - start_time
        avg_time_per_image = elapsed_time / len(image_files) if image_files else 0

        print(f"  => 完成 {file_name} 的所有圖片處理！總耗時: {elapsed_time:.2f} 秒 (平均每張 {avg_time_per_image:.2f} 秒)")

        combined_captions = " | ".join(all_captions)
        output_df.at[index, 'captions'] = combined_captions
        
        # 每處理完一個 file_name 就存檔一次
        output_df.to_csv(output_csv_path, index=False)

    print(f"\n--- {data_type} 資料集處理完成！結果已儲存至: {output_csv_path} ---")


if __name__ == "__main__":
    try:
        main_model, main_processor = load_model_and_processor()
        print("\n*** 開始處理訓練集資料 ***")
        process_dataset(main_model, main_processor, ROAD_TRAIN_CSV, 'road', OUTPUT_ROAD_TRAIN_CSV, is_test_set=False)
        process_dataset(main_model, main_processor, FREEWAY_TRAIN_CSV, 'freeway', OUTPUT_FREEWAY_TRAIN_CSV, is_test_set=False)
    except Exception as e:
        print(f"\n程式執行時發生嚴重錯誤: {e}")

正在載入模型，請稍候...
偵測到 CUDA 設備: NVIDIA GeForce RTX 3060 Ti


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


模型載入完成！

*** 開始處理訓練集資料 ***

--- 開始處理 road (訓練集) ---


處理 road 進度:   0%|                                                                          | 0/179 [00:00<?, ?it/s]

...

  => 完成 freeway_0177 的所有圖片處理！總耗時: 187.11 秒 (平均每張 3.40 秒)
  -> 正在處理 freeway_0178 的第 1/91 張圖片: 00001.jpg
  -> 正在處理 freeway_0178 的第 2/91 張圖片: 00002.jpg
  -> 正在處理 freeway_0178 的第 3/91 張圖片: 00003.jpg
  -> 正在處理 freeway_0178 的第 4/91 張圖片: 00004.jpg
  -> 正在處理 freeway_0178 的第 5/91 張圖片: 00005.jpg
  -> 正在處理 freeway_0178 的第 6/91 張圖片: 00006.jpg
  -> 正在處理 freeway_0178 的第 7/91 張圖片: 00007.jpg
  -> 正在處理 freeway_0178 的第 8/91 張圖片: 00008.jpg
  -> 正在處理 freeway_0178 的第 9/91 張圖片: 00009.jpg
  -> 正在處理 freeway_0178 的第 10/91 張圖片: 00010.jpg
  -> 正在處理 freeway_0178 的第 11/91 張圖片: 00011.jpg
  -> 正在處理 freeway_0178 的第 12/91 張圖片: 00012.jpg
  -> 正在處理 freeway_0178 的第 13/91 張圖片: 00013.jpg
  -> 正在處理 freeway_0178 的第 14/91 張圖片: 00014.jpg
  -> 正在處理 freeway_0178 的第 15/91 張圖片: 00015.jpg
  -> 正在處理 freeway_0178 的第 16/91 張圖片: 00016.jpg
  -> 正在處理 freeway_0178 的第 17/91 張圖片: 00017.jpg
  -> 正在處理 freeway_0178 的第 18/91 張圖片: 00018.jpg
  -> 正在處理 freeway_0178 的第 19/91 張圖片: 00019.jpg
  -> 正在處理 freeway_0178 的第 20/91 張圖片: 00020.jpg
  -> 正在處理 fr

處理 freeway 進度:  99%|████████████████████████████████████████████████████████▋| 179/180 [12:39:27<03:32, 212.52s/it]

  => 完成 freeway_0178 的所有圖片處理！總耗時: 297.89 秒 (平均每張 3.27 秒)
  -> 正在處理 freeway_0179 的第 1/65 張圖片: 00001.jpg
  -> 正在處理 freeway_0179 的第 2/65 張圖片: 00002.jpg
  -> 正在處理 freeway_0179 的第 3/65 張圖片: 00003.jpg
  -> 正在處理 freeway_0179 的第 4/65 張圖片: 00004.jpg
  -> 正在處理 freeway_0179 的第 5/65 張圖片: 00005.jpg
  -> 正在處理 freeway_0179 的第 6/65 張圖片: 00006.jpg
  -> 正在處理 freeway_0179 的第 7/65 張圖片: 00007.jpg
  -> 正在處理 freeway_0179 的第 8/65 張圖片: 00008.jpg
  -> 正在處理 freeway_0179 的第 9/65 張圖片: 00009.jpg
  -> 正在處理 freeway_0179 的第 10/65 張圖片: 00010.jpg
  -> 正在處理 freeway_0179 的第 11/65 張圖片: 00011.jpg
  -> 正在處理 freeway_0179 的第 12/65 張圖片: 00012.jpg
  -> 正在處理 freeway_0179 的第 13/65 張圖片: 00013.jpg
  -> 正在處理 freeway_0179 的第 14/65 張圖片: 00014.jpg
  -> 正在處理 freeway_0179 的第 15/65 張圖片: 00015.jpg
  -> 正在處理 freeway_0179 的第 16/65 張圖片: 00016.jpg
  -> 正在處理 freeway_0179 的第 17/65 張圖片: 00017.jpg
  -> 正在處理 freeway_0179 的第 18/65 張圖片: 00018.jpg
  -> 正在處理 freeway_0179 的第 19/65 張圖片: 00019.jpg
  -> 正在處理 freeway_0179 的第 20/65 張圖片: 00020.jpg
  -> 正在處理 fr

處理 freeway 進度: 100%|█████████████████████████████████████████████████████████| 180/180 [12:43:43<00:00, 254.58s/it]

  => 完成 freeway_0179 的所有圖片處理！總耗時: 256.05 秒 (平均每張 3.94 秒)

--- freeway 資料集處理完成！結果已儲存至: ./freeway_train_with_captions.csv ---
